<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Alan Contreras Montanares


### **Link de repositorio de GitHub:** `https://github.com/AlanMontanares/MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.6 MB 10.9 MB/s eta 0:00:02
   -- ------------------------------------- 1.1/15.6 MB 11.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.6/15.6 MB 11.3 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/15.6 MB 11.2 MB/s eta 0:00:02
   ------ --------------------------------- 2.7/15.6 MB 11.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.2/15.6 MB 11.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/15.6 MB 11.3 MB/s eta 0:00:02
   ---------- ----------------------------- 4.2/15.6 MB 11.3 MB/s eta 0:00:02
   ------------ --------------------------- 4.8/15.6 MB 11.3 MB/s eta 0:00:01
   ------------- -------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players

Name          Nationality National_Position Club_Position  \
0      Cristiano Ronaldo             Portugal                LS            LW   
1           Lionel Messi            Argentina                RW            RW   
2                 Neymar               Brazil                LW            LW   
3            Luis Suárez              Uruguay                LS            ST   
4           Manuel Neuer              Germany                GK            GK   
...                  ...                  ...               ...           ...   
17583        Adam Dunbar  Republic of Ireland               NaN           Sub   
17584       Dylan McGoey  Republic of Ireland               NaN           Sub   
17585    Tommy Ouldridge              England               NaN           Res   
17586         Mark Foden             Scotland               NaN           Sub   
17587   Barry Richardson              England               NaN           Sub   

       Height  Weight Preffered_Foot  Age        Work_Rate  Weak_foot  ...  \
0         185      80          Right   32       High / Low          4  ...   
1         170      72           Left   29  Medium / Medium          4  ...   
2         174      68          Right   25    High / Medium          5  ...   
3         182      85          Right   30    High / Medium          4  ...   
4         193      92          Right   31  Medium / Medium          4  ...   
...       ...     ...            ...  ...              ...        ...  ...   
17583     183      82          Right   19  Medium / Medium          1  ...   
17584     185      80          Right   19  Medium / Medium          2  ...   
17585     173      61          Right   18    High / Medium          2  ...   
17586     180      80          Right   21  Medium / Medium          3  ...   
17587     185      77          Right   47  Medium / Medium          2  ...   

       Agility  Jumping  Heading  Shot_Power  Finishing  Long_Shots  Curve  \
0           90       95       85          92         93          90     81   
1           90       68       71          85         95          88     89   
2           96       61       62          78         89          77     79   
3           86       69       77          87         94          86     86   
4           52       78       25          25         13          16     14   
...        ...      ...      ...         ...        ...         ...    ...   
17583       27       56       14          16         13          13     11   
17584       28       53       12          17         12          11     12   
17585       54       61       41          44         28          42     35   
17586       34       48       15          23         14          12     13   
17587       38       51       12          13         11          16     12   

       Freekick_Accuracy  Penalties  Volleys  
0                     76         85       88  
1                     90         74       85  
2                     84         81       83  
3                     84         85       88  
4                     11         47       11  
...                  ...        ...      ...  
17583                 13         15       12  
17584                 13         16       12  
17585                 36         42       37  
17586                 12         24       12  
17587                 11         22       12  

[17588 rows x 39 columns]

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [3]:
df_players["National_Position"].value_counts(dropna=False)

National_Position
NaN    16513
Sub      556
LCB       48
GK        47
RCB       46
LB        39
RB        38
RM        34
LM        32
ST        30
LCM       25
RCM       25
LDM       19
CAM       19
RDM       18
RS        18
LS        18
CB         9
CDM        9
CM         9
RW         7
LW         7
LWB        4
RAM        4
LAM        4
RWB        4
RF         3
LF         3
Name: count, dtype: int64

In [4]:
labels = df_players["National_Position"].apply(lambda x: 1 if not pd.isna(x) else 0)
labels.value_counts()

National_Position
0    16513
1     1075
Name: count, dtype: int64

In [5]:
df_players = df_players.drop("National_Position", axis=1)

Si los que tienen valor Nan son aquellos no seleccionados mientras que el resto si esta seleccionado entonces tendremos 16.513 no seleccionados y 1075 seleccionados, dando un gran desbalance entre clases.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [6]:
df_players.shape

(17588, 38)

En este caso normalizaremos las columnas numericas con Min-Max y las categoricas las encodearemos con OneHotEncoder. Cabe recalcar que se podria usar LabelEncoder, en ese caso tambien es recomendable normalizar con Min-Max o StandardScaler para no tener problemas con algunos algoritmos.

In [7]:
numeric_features = df_players.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df_players.select_dtypes(include=['object']).columns


preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_features), 
    ],
    remainder='passthrough'  
)

Respecto a los modelos, segun la guia como tenemos menos de 100.000 ejemplos se recomienda usar (en orden), LinearSVC(), KNeighborsClassifier() y algun modelo de ensamble, en este caso usaremos RandomForest.

In [8]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC(dual='auto'))
])

pipeline_kneighbors = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

Separamos en 70% para entrenar y 30% para test conservando la proporcion de clases con stratity.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_players, labels.values, test_size=0.3, random_state=42, stratify=labels.values)

In [10]:
pipeline_svc.fit(X_train, y_train)
svc_predictions = pipeline_svc.predict(X_test)
print(classification_report(y_test, svc_predictions))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      4954
           1       0.70      0.34      0.46       323

    accuracy                           0.95      5277
   macro avg       0.83      0.67      0.72      5277
weighted avg       0.94      0.95      0.94      5277



In [11]:
pipeline_kneighbors.fit(X_train, y_train)
kn_predictions = pipeline_kneighbors.predict(X_test)
print(classification_report(y_test, kn_predictions))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.54      0.08      0.14       323

    accuracy                           0.94      5277
   macro avg       0.74      0.54      0.56      5277
weighted avg       0.92      0.94      0.92      5277



In [12]:
pipeline_rf.fit(X_train, y_train)
rf_predictions = pipeline_rf.predict(X_test)
print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.92      0.04      0.07       323

    accuracy                           0.94      5277
   macro avg       0.93      0.52      0.52      5277
weighted avg       0.94      0.94      0.91      5277



**Respuesta**:

```
Para todos los modelos se realizo una buena clasificacion de la clase negativa (0), los cuales no son seleccionados, pero una mala prediccion para la clase positiva. En este caso se puede ver que la presicion es muy buena en ambas clases, sin embargo esta metrica no debe ser usada cuando las clases estan desbalanceadas, ya que se podria esta subestimando o sobreestimando los resultados, agregando de que el recall de la clase negativa fue de un 100% por lo que no habrian falsos negativos, dando por efecto que la precision de la clase positiva fuera alta, de hecho, con solo 1 prediccion correcta tendria una precision del 100%. Como el F1-score depende de la precision, tampoco es buena idea tomarla en cuenta por los mismos motivos

Respecto al recall se puede ver que todas las clases negativas en la mayoria de metricas fueron predichas de forma correcta, no asi con la clase positiva que tiene un pesimo recall (el metodo SVC tiene mejores metricas pero igualemente son malos resultados). En caso de desbalance en el set de test, esta es una buena metrica a considerar ya que no depende de este problema, ya que se evalua la cantidad de predicciones correctas de cada clase de forma independiente, de hecho el accuracy equivale al recall en terminos micros.

A modo de resumen el accuracy y el micro recall son la tasa de predicciones correctas en una clase determinada, mientras que la presicion se podria relacionar con que tanto el modelo se confunde entre las clases. Por ejemplo, si tenemos una presicion de 5% para la clase positiva, equivale a decir que si el modelo predice un imput como clase positiva es muy improbable que sea verdad, por otro lado tener una presicion del 100% en la clase positiva equivaldria a tener la certeza (aunque siempre hay casos y casos) de que si el modelo dice que el input es de la clase positiva es seguro. En este caso las clases estan desbalanceadas por lo que la presicion no es del todo realista, por lo que no se podria realizar esta afirmacion.

Si se podrian mejorar las metricas, en donde algo importante son los hiperparametros, en este caso se tienen los hiperparametros por defecto pero se podria usar cross-validation para encontrar los mejores (y evitar posible overfitting). Asi tambien, se podria realizar feature selection, ya que podria ser que no todas las caracteristicas utilizadas sean informativas para el problema. Por ultimo se podria tratar de usar LabelEncoder(), debido a que al usar OneHotEncoder() queda una matriz sparse, y metodos como RandomForest toman una porcion de caracteristicas al azar, por lo que es mayor la probabilidad de que tome caracteristicas que no importen.

Siempre es recomendable usar datos balanceados, tanto en problemas supervisados como no. En este caso seria ideal trabajar con clases balanceadas, ya que los modelos son mas capaces de ajustarse a las clases mayoritarias. De hecho, en este caso se obtuvieron mejores resultados en la clase con mas ejemplos (negativa).

```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [13]:
df_players_copy = df_players.copy()

df_players_copy["Club_Position"].replace(("ST", "CF"),"ataque", inplace=True)
df_players_copy["Club_Position"].replace(("RW", "CAM","LW"),"central_ataque", inplace=True)
df_players_copy["Club_Position"].replace(("RM", "CM","LM"),"central", inplace=True)
df_players_copy["Club_Position"].replace(("RWB", "CDM","LWB"),"central_defensa", inplace=True)
df_players_copy["Club_Position"].replace(("RB", "CB","LB"),"defensa", inplace=True)
df_players_copy["Club_Position"].replace("GK","arquero", inplace=True)

df_players_copy = df_players_copy[df_players_copy["Club_Position"].isin(["ataque", "central_ataque", "central", "central_defensa", "defensa", "arquero"])]

df_players_copy["Club_Position"].value_counts()

Club_Position
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64

In [14]:
y = df_players_copy['Club_Position']
X = df_players_copy.drop('Club_Position', axis=1)

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_features), 
    ],
    remainder='passthrough')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Los datos que se ocupan son los mismos solo que el target es distinto y hay menos muestras, por lo que los clasificadores se conservan.

In [16]:
pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC(dual='auto'))
])

pipeline_kneighbors = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [17]:
pipeline_svc.fit(X_train, y_train)
svc_predictions = pipeline_svc.predict(X_test)
print(classification_report(y_test, svc_predictions))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.74      0.81      0.78       129
        central       0.59      0.59      0.59       272
 central_ataque       0.50      0.36      0.42       174
central_defensa       0.43      0.16      0.23        63
        defensa       0.80      0.98      0.88       354

       accuracy                           0.74      1182
      macro avg       0.68      0.65      0.65      1182
   weighted avg       0.71      0.74      0.72      1182



In [18]:
pipeline_kneighbors.fit(X_train, y_train)
kn_predictions = pipeline_kneighbors.predict(X_test)
print(classification_report(y_test, kn_predictions))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.52      0.51      0.51       129
        central       0.47      0.56      0.51       272
 central_ataque       0.36      0.24      0.28       174
central_defensa       0.29      0.16      0.21        63
        defensa       0.73      0.81      0.77       354

       accuracy                           0.63      1182
      macro avg       0.56      0.54      0.55      1182
   weighted avg       0.61      0.63      0.62      1182



In [19]:
pipeline_rf.fit(X_train, y_train)
rf_predictions = pipeline_rf.predict(X_test)
print(classification_report(y_test, rf_predictions))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.75      0.80      0.77       129
        central       0.55      0.65      0.60       272
 central_ataque       0.44      0.21      0.28       174
central_defensa       0.43      0.05      0.09        63
        defensa       0.78      0.98      0.87       354

       accuracy                           0.72      1182
      macro avg       0.66      0.61      0.60      1182
   weighted avg       0.69      0.72      0.69      1182



**Respuesta**:

```
En este caso se obtuvieron buenos resultados de forma general con LinealSVC(), sin embarbo cabe recalcar que hubieron clases que los modelos no pudieron predecir bien, como la clase "central_defensa" y "central_ataque". Por otro lado, se podria pensar que los modelos se ajustarian a las clases mayoritarias como "defensa" o "central", en donde se obtuvieron metricas decentes, sin embargo la clase "arquero" fue la que mejor se predice en todos los modelos.

Esto podria darse en que hay caracteristicas muy informativas para esa clase, como los penaltis que tira, el dribbling, etc, los cuales son poco comunes en arqueros (que es lo que usualmente se piensa), sin embargo no todos los escenarios son los mismos y estas variables podrian no ser utiles, por lo que habria que realizar un analisis mas a fondo de las caracteristicas y sus relaciones con el target.

Nuevamente, los resultados podrian mejorar realizando feature selection, cross-validation y un balance entre clases. Tambien se podria probar con aplicar LabelEncoder() dadas las mismas razones que el item anterior.

```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [20]:
salarios_df = pd.read_csv('salarios.csv')
salarios_df.head()

Unnamed: 0             Player  Weekly Salary
0        1528       Lionel Messi      1538905.0
1        1006  Cristiano Ronaldo      1248536.0
2        1529  Antoine Griezmann       997032.0
3        2004             Neymar       797726.0
4        1530        Gareth Bale       607706.0

In [21]:
df_players = pd.read_csv('stats_players.csv')
df_players.head()

Name Nationality National_Position Club_Position  Height  \
0  Cristiano Ronaldo    Portugal                LS            LW     185   
1       Lionel Messi   Argentina                RW            RW     170   
2             Neymar      Brazil                LW            LW     174   
3        Luis Suárez     Uruguay                LS            ST     182   
4       Manuel Neuer     Germany                GK            GK     193   

   Weight Preffered_Foot  Age        Work_Rate  Weak_foot  ...  Agility  \
0      80          Right   32       High / Low          4  ...       90   
1      72           Left   29  Medium / Medium          4  ...       90   
2      68          Right   25    High / Medium          5  ...       96   
3      85          Right   30    High / Medium          4  ...       86   
4      92          Right   31  Medium / Medium          4  ...       52   

   Jumping  Heading  Shot_Power  Finishing  Long_Shots  Curve  \
0       95       85          92         93          90     81   
1       68       71          85         95          88     89   
2       61       62          78         89          77     79   
3       69       77          87         94          86     86   
4       78       25          25         13          16     14   

   Freekick_Accuracy  Penalties  Volleys  
0                 76         85       88  
1                 90         74       85  
2                 84         81       83  
3                 84         85       88  
4                 11         47       11  

[5 rows x 39 columns]

In [22]:
salarios_df.shape

(2534, 3)

In [23]:
df_players.shape

(17588, 39)

In [24]:
df_interseccion = pd.merge(df_players, salarios_df, left_on='Name', right_on='Player', how='inner') #Realizamos la interseccion respecto a los nombres
df_interseccion = df_interseccion.drop(['Player', 'Name','Unnamed: 0'], axis=1) #Eliminamos columnas que no usaremos

df_interseccion["National_Position"] = df_interseccion["National_Position"].apply(lambda x: 1 if not pd.isna(x) else 0) # Mapeamos si es seleccionado o no

df_interseccion.head()

Nationality  National_Position Club_Position  Height  Weight Preffered_Foot  \
0    Portugal                  1            LW     185      80          Right   
1   Argentina                  1            RW     170      72           Left   
2      Brazil                  1            LW     174      68          Right   
3     Uruguay                  1            ST     182      85          Right   
4     Germany                  1            GK     193      92          Right   

   Age        Work_Rate  Weak_foot  Skill_Moves  ...  Jumping  Heading  \
0   32       High / Low          4            5  ...       95       85   
1   29  Medium / Medium          4            4  ...       68       71   
2   25    High / Medium          5            5  ...       61       62   
3   30    High / Medium          4            4  ...       69       77   
4   31  Medium / Medium          4            1  ...       78       25   

   Shot_Power  Finishing  Long_Shots  Curve  Freekick_Accuracy  Penalties  \
0          92         93          90     81                 76         85   
1          85         95          88     89                 90         74   
2          78         89          77     79                 84         81   
3          87         94          86     86                 84         85   
4          25         13          16     14                 11         47   

   Volleys  Weekly Salary  
0       88      1248536.0  
1       85      1538905.0  
2       83       797726.0  
3       88       508923.0  
4       11       326233.0  

[5 rows x 39 columns]

In [42]:
df_numerico = df_interseccion.select_dtypes(include='number') #Usamos solo columnas numericas

y = df_numerico['Weekly Salary']
X = df_numerico.drop('Weekly Salary', axis=1)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), X.columns),
    ],
    remainder='passthrough')

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
from sklearn.ensemble import RandomForestRegressor

pipeline_regresion = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor())
])

In [47]:
from sklearn.metrics import r2_score

pipeline_regresion.fit(X_train, y_train)

regresion_predictions = pipeline_regresion.predict(X_test)

r2 = r2_score(y_test, regresion_predictions)
print(r2)

0.41257887036719565


**Respuesta**:

```

En resumen, la metrica R2 mide en cierta medida la variacion entre la regresion creada respecto al promedio del target como tal, o en otras palabras, indica qué tan bien el modelo se ajusta a los datos observados. Esta metrica se encuentra en el rango de 0 a 1, donde 0 significa que la variable independiente no explica nada de la variabilidad en la variable dependiente, por lo que el modelo es inútil para hacer predicciones. Por otro lado, cuando R2 es igual a 1, significa que todas las variaciones en la variable dependiente (target) pueden explicarse completamente por la variable independiente, por lo que el modelo hace predicciones perfectas (aunque este número por sí solo no puede determinar si los puntos de datos o las predicciones están sesgados).

Su formula esta dada por: R2 = 1- (var(res)/var(media)). Donde var(res) es la suma de los residuos respecto a la regresion creada y var(media) es respecto a la media de la variable a predecir (target).

Respecto a los resultados, se probó en primera instancia usar todos los datos (numericos y categoricos) y luego solo los numericos, en donde se obtuvieron valores similares, por lo que solo se mostrara el resultado al usar solo datos numericos.

La razon de esto es porque se intento realizar la regresion eliminando en primera instancia el nombre del jugador, y estimar su salario unicamente en las caracteristicas fisicas y de juego de este. Si hubieramos usado el nombre como una caracteristica probablemente el modelo se hubiera ajustado a esto, y no es lo que queremos ya que el nombre no influye en la ganancia (o quizas si en la vida real, con nombres como Messi o el bicho siuuu). Es por eso que unicamente intentamos predecir el salario unicamente en base a las caracteristicas fisicas y de juego.

Ahora bien, se intento con una regresion lineal basica, en donde se obtuvo un R2 por debajo de lo que se pedia. Dicho esto, se uso directamente RandomForestRegressor(), si bien la guia recomendaba primero usar la regression Lasso, se hizo este salto para probar directamente con este modelo el cual arrojo un R2 cercano a 0.41, lo cual es mayor a lo que se pedia.

Nuevamente, para mejorar los resultados se podria hace feature selection, cross-validation, usar LabelEncoder() y tambien seria ideal poder tener mas caracteristicas importantes para el problema y tambien aumentar la cantidad de datos.

La razon de obtener mas caracteristicas es porque con las actuales no es posible obtener tan buenos resultados, aunque se podria realizar un red de regresion con no-linearidades pero no es lo ideal ya que igual se perderia interpretacion. Respecto a tener mas datos se dice porque en general los sueldos tienen una distribucion, en donde el modelo por lo general se adecua donde se concentra esta distribucion (similar al desbalance entre clases), por lo que seria ideal tener un dataset con una distribucion más plana para que no existan sesgos en las predicciones.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>